In [1]:
import scanpy as sc

import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
from scib_metrics.benchmark import Benchmarker

from scdataloader import utils as data_utils


from collections import Counter

%load_ext autoreload
%autoreload 2

import torch
torch.set_float32_matmul_precision('medium')


→ connected lamindb: jkobject/scprint


In [2]:
import shutil
import os

from geneformer import EmbExtractor
from geneformer import TranscriptomeTokenizer
from grnndata import GRNAnnData

from geneformer import TOKEN_DICTIONARY_FILE
import pickle

with open(TOKEN_DICTIONARY_FILE, "rb") as f:
    gene_mapping_dict = pickle.load(f)

def compute_geneformer_embs(adata, model="../../Geneformer/fine_tuned_models/gf-12L-95M-i4096_MTLCellClassifier_CELLxGENE_240522/", model_type="CellClassifier", forward_batch_size=40, max_ncells=100_000, cache=False):
    if not (cache and os.path.exists("/tmp/geneformer/test.h5ad")):
        adata = adata[:, adata.var.index.isin([u for u in gene_mapping_dict.keys() if u in adata.var.index])]

        adata.var['ensembl_id'] = adata.var.index
        adata.obs['n_counts'] = adata.X.sum(1)
        # Create the geneformer folder if it doesn't exist
        geneformer_folder = "/tmp/geneformer"
        if not os.path.exists(geneformer_folder):
            os.makedirs(geneformer_folder)
        adata.write_h5ad("/tmp/geneformer/test.h5ad")
        tokenized_data_path = "/tmp/geneformer/tokenized_data.dataset/"
        if os.path.exists(tokenized_data_path):
            shutil.rmtree(tokenized_data_path)
        tk = TranscriptomeTokenizer({"cell_type": "cell_type"}, nproc=16, model_input_size=4096)
        tk.tokenize_data("/tmp/geneformer/", 
                        "/tmp/geneformer/",
                        "tokenized_data", 
                        file_format="h5ad")
    else:
        adata = sc.read("/tmp/geneformer/test.h5ad")

    genelist = [gene_mapping_dict[u] for u in adata.var.index]

    embex = EmbExtractor(model_type=model_type, #CellClassifier
                     max_ncells=max_ncells,
                     emb_mode="cell",
                     forward_batch_size=forward_batch_size,
                     nproc=16,
                     genelist=genelist)
    embs, avg_attentions = embex.extract_embs(model, #../../../Geneformer/fine_tuned_models/gf-6L-30M-i2048_CellClassifier_cardiomyopathies_220224/
                          "/tmp/geneformer/tokenized_data.dataset/",
                          "/tmp/geneformer",
                          "geneformertest",
                          get_avg_attentions=False,
                          )
    return embs

# pancreas

In [3]:
adata = sc.read(
    "data/pancreas_atlas.h5ad",
    backup_url="https://figshare.com/ndownloader/files/24539828",
)


In [4]:
# preprocess

# Merge adata.var with genesdf
# First, ensure the index of adata.var is named
adata.obs['organism_ontology_term_id'] = "NCBITaxon:9606"
genesdf = data_utils.load_genes(adata.obs.organism_ontology_term_id.iloc[0])
adata.var.index.name = 'symbol'

# Merge adata.var with genesdf
# Merge adata.var with genesdf, keeping only the first instance of each symbol
merged_var = adata.var.reset_index().merge(
    genesdf.reset_index().drop_duplicates(subset='symbol', keep='first'),
    left_on='symbol',
    right_on='symbol',
    how='left'
)

# Set the index back to 'symbol'
merged_var.set_index('symbol', inplace=True)
adata.var = merged_var
adata.var['symbol'] =  adata.var.index
adata.var.index = adata.var.ensembl_gene_id
adata.var.drop(columns=['mt', 'ribo', 'hb'], inplace=True)

In [5]:
adata.X = adata.layers['counts']
del adata.layers['counts']

In [7]:
embs = compute_geneformer_embs(adata, forward_batch_size=20, max_ncells=100_000, cache=False)
adata.obsm['geneformer'] = embs.values

/tmp/ipykernel_2629171/19730043.py:18: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['ensembl_id'] = adata.var.index


Tokenizing /tmp/geneformer/test.h5ad


/home/ml4ig1/Documents code/Geneformer/geneformer/tokenizer.py:473: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  for i in adata.var["ensembl_id_collapsed"][coding_miRNA_loc]
/home/ml4ig1/Documents code/Geneformer/geneformer/tokenizer.py:476: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coding_miRNA_ids = adata.var["ensembl_id_collapsed"][coding_miRNA_loc]


/tmp/geneformer/test.h5ad has no column attribute 'filter_pass'; tokenizing all cells.
Creating dataset.


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../../Geneformer/fine_tuned_models/gf-12L-95M-i4096_MTLCellClassifier_CELLxGENE_240522/ and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[d3graph] WARNING> CLS token present in token dictionary, excluding from average.
[d3graph] WARNING> EOS token present in token dictionary, excluding from average.


  0%|          | 0/820 [00:00<?, ?it/s]

In [8]:
adata.obsm['geneformer'] = embs.values

In [9]:
sc.pp.pca(adata)

In [10]:
bm = Benchmarker(
    adata,
    batch_key="tech", #batch, tech
    label_key="celltype", #celltype
    embedding_obsm_keys=["X_pca", "geneformer"],
    n_jobs=20,
)
bm.benchmark()

/home/ml4ig1/miniconda3/envs/geneformer/lib/python3.10/site-packages/scanpy/preprocessing/_pca.py:377: FutureWarning: Argument `use_highly_variable` is deprecated, consider using the mask argument. Use_highly_variable=True can be called through mask_var="highly_variable". Use_highly_variable=False can be called through mask_var=None
  warn(msg, FutureWarning)
Embeddings:   0%|          | 0/2 [00:00<?, ?it/s][d3graph] WARNING> An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
/home/ml4ig1/miniconda3/envs/geneformer/lib/python3.10/site-packages/scib_metrics/_kbet.py:181: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  comp_size = pd.value_counts(labs)
/home/ml4ig1/miniconda3/envs/geneformer/lib/python3.10/site-packages/scib_metrics/_kbet.py:181: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.S

INFO     t_cell consists of a single batch or is too small. Skip.                                                  


/home/ml4ig1/miniconda3/envs/geneformer/lib/python3.10/site-packages/scib_metrics/_kbet.py:181: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  comp_size = pd.value_counts(labs)
/home/ml4ig1/miniconda3/envs/geneformer/lib/python3.10/site-packages/scib_metrics/_graph_connectivity.py:30: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  tab = pd.value_counts(comps)
Embeddings:  50%|█████     | 1/2 [01:30<01:30, 90.22s/it]/home/ml4ig1/miniconda3/envs/geneformer/lib/python3.10/site-packages/scib_metrics/_kbet.py:181: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  comp_size = pd.value_counts(labs)
/home/ml4ig1/miniconda3/envs/geneformer/lib/python3.10/site-packages/scib_metrics/_kbet.py:181: FutureWarning: pandas.value_counts is deprec

INFO     t_cell consists of a single batch or is too small. Skip.                                                  


/home/ml4ig1/miniconda3/envs/geneformer/lib/python3.10/site-packages/scib_metrics/_kbet.py:181: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  comp_size = pd.value_counts(labs)/home/ml4ig1/miniconda3/envs/geneformer/lib/python3.10/site-packages/scib_metrics/_graph_connectivity.py:30: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  tab = pd.value_counts(comps)
Embeddings: 100%|██████████| 2/2 [07:37<00:00, 228.74s/it]


In [11]:
# pancreas
bm.get_results(min_max_scale=False)

,Isolated labels,KMeans NMI,KMeans ARI,Silhouette label,cLISI,Silhouette batch,iLISI,KBET,Graph connectivity,PCR comparison,Batch correction,Bio conservation,Total
Embedding,,,,,,,,,,,,,
X_pca,0.986439,0.169171,-0.036007,0.14008,0.999789,0.723848,0.064154,0.125128,0.643397,0.0,0.311305,0.451894,0.395659
geneformer,0.457926,0.063174,0.023233,0.469323,0.828791,0.869874,0.191862,0.007704,0.168072,0.633788,0.37426,0.368489,0.370798
Metric Type,Bio conservation,Bio conservation,Bio conservation,Bio conservation,Bio conservation,Batch correction,Batch correction,Batch correction,Batch correction,Batch correction,Aggregate score,Aggregate score,Aggregate score


In [4]:
import datetime
from geneformer import Classifier

In [6]:
adata = adata[:, adata.var.index.isin([u for u in gene_mapping_dict.keys() if u in adata.var.index])]

adata.var['ensembl_id'] = adata.var.index
adata.obs['n_counts'] = adata.X.sum(1)
# Create the geneformer folder if it doesn't exist
geneformer_folder = "/tmp/geneformer"
if not os.path.exists(geneformer_folder):
    os.makedirs(geneformer_folder)
adata.write_h5ad("/tmp/geneformer/test.h5ad")
tokenized_data_path = "/tmp/geneformer/tokenized_data.dataset/"
if os.path.exists(tokenized_data_path):
    shutil.rmtree(tokenized_data_path)
tk = TranscriptomeTokenizer({"celltype": "cell_type"}, nproc=16, model_input_size=4096)
tk.tokenize_data("/tmp/geneformer/", 
                "/tmp/geneformer/",
                "tokenized_data", 
                file_format="h5ad")

/tmp/ipykernel_341724/1695375894.py:3: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['ensembl_id'] = adata.var.index


Tokenizing /tmp/geneformer/test.h5ad


/home/ml4ig1/Documents code/Geneformer/geneformer/tokenizer.py:473: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  for i in adata.var["ensembl_id_collapsed"][coding_miRNA_loc]
/home/ml4ig1/Documents code/Geneformer/geneformer/tokenizer.py:476: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coding_miRNA_ids = adata.var["ensembl_id_collapsed"][coding_miRNA_loc]


/tmp/geneformer/test.h5ad has no column attribute 'filter_pass'; tokenizing all cells.
Creating dataset.


In [7]:
from transformers import BertForSequenceClassification
from transformers import Trainer


In [9]:
ft_model = BertForSequenceClassification.from_pretrained("../../Geneformer/fine_tuned_models/gf-12L-95M-i4096_MTLCellClassifier_CELLxGENE_240522/")
ft_trainer = Trainer(model=ft_model)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../../Geneformer/fine_tuned_models/gf-12L-95M-i4096_MTLCellClassifier_CELLxGENE_240522/ and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# 2. load new dataset
import pandas as pd
from datasets import Dataset, load_from_disk
new_dataset = load_from_disk("/tmp/geneformer/tokenized_data.dataset/")
pd.DataFrame(new_dataset)

,input_ids,cell_type,length
0,"[2, 3569, 12932, 3515, 2338, 2613, 11773, 1738...",gamma,1757
1,"[2, 3569, 18360, 1738, 14358, 11773, 1819, 184...",gamma,3496
2,"[2, 3569, 1738, 3637, 1819, 2338, 11773, 1711,...",gamma,2124
3,"[2, 3569, 18360, 1738, 11773, 11330, 14358, 63...",gamma,2469
4,"[2, 3569, 1731, 16905, 13191, 11452, 1711, 721...",gamma,2090
...,...,...,...
16377,"[2, 3569, 1819, 3637, 11773, 12932, 13233, 150...",gamma,4096
16378,"[2, 3569, 12932, 1819, 11773, 3637, 5946, 1758...",gamma,4096
16379,"[2, 3569, 11122, 4753, 1819, 12932, 12465, 171...",gamma,4096
16380,"[2, 4753, 3569, 11773, 5232, 6635, 7505, 5045,...",gamma,3842


In [12]:
ct_predictions = ft_trainer.predict(new_dataset)
ct_pred = ct_predictions.predictions

ValueError: expected sequence of length 1757 at dim 1 (got 3496)

In [ ]:
# it seems we can't just do inference even with the fine tuned model... 

# lung

In [3]:
adata = sc.read(
    "data/lung_atlas.h5ad",
    backup_url="https://figshare.com/ndownloader/files/24539942",
)

In [4]:
# preprocess

# Merge adata.var with genesdf
# First, ensure the index of adata.var is named
adata.obs['organism_ontology_term_id'] = "NCBITaxon:9606"
genesdf = data_utils.load_genes(adata.obs.organism_ontology_term_id.iloc[0])
adata.var.index.name = 'symbol'

# Merge adata.var with genesdf
# Merge adata.var with genesdf, keeping only the first instance of each symbol
merged_var = adata.var.reset_index().merge(
    genesdf.reset_index().drop_duplicates(subset='symbol', keep='first'),
    left_on='symbol',
    right_on='symbol',
    how='left'
)

# Set the index back to 'symbol'
merged_var.set_index('symbol', inplace=True)
adata.var = merged_var
adata.var['symbol'] =  adata.var.index
adata.var.index = adata.var.ensembl_gene_id
adata.var.drop(columns=['mt', 'ribo', 'hb'], inplace=True)

In [5]:
adata.X = adata.layers['counts']
del adata.layers['counts']

In [6]:
embs = compute_geneformer_embs(adata, forward_batch_size=20, max_ncells=100_000, cache=False)
adata.obsm['geneformer'] = embs.values

/tmp/ipykernel_2703648/19730043.py:18: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['ensembl_id'] = adata.var.index


Tokenizing /tmp/geneformer/test.h5ad


/home/ml4ig1/Documents code/Geneformer/geneformer/tokenizer.py:473: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  for i in adata.var["ensembl_id_collapsed"][coding_miRNA_loc]
/home/ml4ig1/Documents code/Geneformer/geneformer/tokenizer.py:476: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coding_miRNA_ids = adata.var["ensembl_id_collapsed"][coding_miRNA_loc]


/tmp/geneformer/test.h5ad has no column attribute 'filter_pass'; tokenizing all cells.
Creating dataset.


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../../Geneformer/fine_tuned_models/gf-12L-95M-i4096_MTLCellClassifier_CELLxGENE_240522/ and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[d3graph] WARNING> CLS token present in token dictionary, excluding from average.
[d3graph] WARNING> EOS token present in token dictionary, excluding from average.


  0%|          | 0/1624 [00:00<?, ?it/s]

In [7]:
sc.pp.pca(adata)

In [8]:
bm = Benchmarker(
    adata,
    batch_key="batch", #batch, tech
    label_key="cell_type", #celltype
    embedding_obsm_keys=["X_pca", "geneformer"],
    n_jobs=20,
)
bm.benchmark()

/home/ml4ig1/miniconda3/envs/geneformer/lib/python3.10/site-packages/scanpy/preprocessing/_pca.py:377: FutureWarning: Argument `use_highly_variable` is deprecated, consider using the mask argument. Use_highly_variable=True can be called through mask_var="highly_variable". Use_highly_variable=False can be called through mask_var=None
  warn(msg, FutureWarning)
Embeddings:   0%|          | 0/2 [00:00<?, ?it/s][d3graph] WARNING> An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
/home/ml4ig1/miniconda3/envs/geneformer/lib/python3.10/site-packages/scib_metrics/_kbet.py:181: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  comp_size = pd.value_counts(labs)
/home/ml4ig1/miniconda3/envs/geneformer/lib/python3.10/site-packages/scib_metrics/_kbet.py:181: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.S

In [9]:
# lung
bm.get_results(min_max_scale=False)

,Isolated labels,KMeans NMI,KMeans ARI,Silhouette label,cLISI,Silhouette batch,iLISI,KBET,Graph connectivity,PCR comparison,Batch correction,Bio conservation,Total
Embedding,,,,,,,,,,,,,
X_pca,0.469209,0.440155,0.236299,0.437615,0.999656,0.820083,0.00932,0.22938,0.766715,0.0,0.3651,0.516587,0.455992
geneformer,0.498888,0.13073,0.052578,0.45876,0.818367,0.906663,0.215395,0.436513,0.17932,0,0.347578,0.391864,0.37415
Metric Type,Bio conservation,Bio conservation,Bio conservation,Bio conservation,Bio conservation,Batch correction,Batch correction,Batch correction,Batch correction,Batch correction,Aggregate score,Aggregate score,Aggregate score


In [ ]:
fine = {"gamma": "CL:0002275",
"beta": "CL:0000169", # "CL:0008024"
"epsilon": "CL:0005019", # "CL:0008024"
"acinar": "CL:0000622",
'delta': 'CL:0000173', #"CL:0008024"
'schwann': 'CL:0002573', # "CL:0000125"
'activated_stellate': 'CL:0000057',
'alpha': 'CL:0000171',# "CL:0008024"
'mast': 'CL:0000097',
'Mast cell': 'CL:0000097',
'quiescent_stellate': 'CL:0000057',
't_cell': 'CL:0000084',
'endothelial': 'CL:0000115',
'Endothelium': 'CL:0000115',
'ductal': 'CL:0002079', #CL:0000068
'macrophage': 'CL:0000235',
'Macrophage': 'CL:0000235',
'B cell': 'CL:0000236',
'Type 2': 'CL:0002063',
'Type 1': 'CL:0002062',
'Ciliated': 'CL:4030034', # respiratory ciliated
'Dendritic cell': 'CL:0000451', #leukocyte
'Secretory': 'CL:0000151',
'Ionocytes': 'CL:0005006',
'Basal 1': 'CL:0000646', # epithelial
'Basal 2': 'CL:0000646',
'Neutrophil_CD14_high': 'CL:0000775',
'Neutrophils_IL1R2': 'CL:0000775',
'Lymphatic': 'CL:0002138',
'Fibroblast': 'CL:0000057',
'T/NK cell': 'CL:0000814',

'inDrop1': 'EFO:0008780',
'inDrop3': 'EFO:0008780',
'inDrop4': 'EFO:0008780',
'inDrop2': 'EFO:0008780',
'fluidigmc1': 'EFO:0010058', #fluidigm c1
'smarter': 'EFO:0010058', #fluidigm c1
'celseq2': 'EFO:0010010',
'smartseq2': 'EFO:0008931',
'celseq': 'EFO:0008679',
}
coarse = {
"beta": "CL:0008024", #endocrine
"epsilon": "CL:0008024",
'delta': "CL:0008024",
'alpha': "CL:0008024",
"gamma": "CL:0008024",
"acinar": "CL:0000150", #epithelial (gland)
'ductal': "CL:0000068", #epithelial (duct)
'schwann': "CL:0000125", # glial
'endothelial': 'CL:0000115',
'Endothelium': 'CL:0000115',
'Lymphatic': 'CL:0000115',
'macrophage': 'CL:0000235', #myeloid leukocyte (not) 
'Macrophage': 'CL:0000235', # myeloid leukocyte
'mast': 'CL:0000097', #myeloid leukocyte (not)
'Mast cell': 'CL:0000097', #myeloid leukocyte
'Neutrophil_CD14_high': 'CL:0000775', #myeloid leukocyte
'Neutrophils_IL1R2': 'CL:0000775', #myeloid leukocyte
't_cell': 'CL:0000084', #leukocyte, lymphocyte (not)
'T/NK cell': 'CL:0000084', #leukocyte, lymphocyte (not)
'B cell': 'CL:0000236', #leukocyte, lymphocyte (not)
'Dendritic cell': 'CL:0000451', #leukocyte, lymphocyte
'activated_stellate': 'CL:0000057', # fibroblast (not)
'quiescent_stellate': 'CL:0000057', # fibroblast (not)
'Fibroblast': 'CL:0000057',
'Type 2': 'CL:0000066', # epithelial
'Type 1': 'CL:0000066',
'Ionocytes': 'CL:0000066', # epithelial
'Basal 1': 'CL:0000066', # epithelial
'Basal 2': 'CL:0000066',
'Ciliated': 'CL:0000064', # ciliated
'Secretory': 'CL:0000151',

'inDrop1': 'EFO:0008780',
'inDrop3': 'EFO:0008780',
'inDrop4': 'EFO:0008780',
'inDrop2': 'EFO:0008780',
'fluidigmc1': 'EFO:0010058', #fluidigm c1
'smarter': 'EFO:0010058', #fluidigm c1
'celseq2': 'EFO:0010010',
'smartseq2': 'EFO:0008931',
'celseq': 'EFO:0008679',
}
